# **PostgreSQL XID**


- PostgreSQL 에서 하나의 튜플은 xmin, xmax 의 2개의 트랜잭션 ID 를 가짐.

  - xmin: 튜플이 Insert 될 때 이후 트랜잭션이볼 수 있도록 가지는 Id[Tuple 이 생성된 시점]
  - xmax: Update, Delete 가 될 때 해당 시점 Id[Tuple 이 삭제되는 (이전의 값) 인 경우]


- 이때 다른 세션에서 해당 데이터를 조회 시 사용되는 트랜잭션ID 가 xmin 과 xmax 사이라면 보여지고, 그렇지 않다면 보여지지 않도록 함

- min ~ max 사이의 트랜잭션 ID 를 가진 친구만, 데이터를 볼 수 있고, 그 이외의 상황에선 데이터를 볼 수 없음.



**PostgreSQL 의 XID 제약조건**

- TID 는 40억개를 사용할 수 있으며 20억개는 older , 20억개는 newer 로 사용.
- 다 쓰면 ID 가 1이 됨.
  - 이렇게 될 경우, Id 가 1인 친구는, xmin 40억인 친구가 자신보다 크기에, 조회할 수가 없다.
  

- 이런 경우를 방지하기 위해 해당 tuple 에 freeze 표기해 무조건 older 로 볼 수 있도록 변경
 - 이렇게 표기된 tuple 은 항상 어떤 트랜잭션에서도 표기되도록 설정해 줌.



**XID 측면에서의 vaccum**


```
autovacuum_freeze_max_age = 1000

vacuum_freeze_min_age = 100
```

- 만약 특정 테이블의 age 가 1000 에 도달하게 된다면, (xid 가 1 증가할때마다 age 도 1 씩 증가) 최근 100 개의 xid 로그를 남겨두고, 나머지는 삭제.[이후 age 는 100 이 됨]
  → 100 개 이전의 xid 에서 생성된 table 처럼 됨

- **relfrozenxid: vacuum 작업을 수행한 xid (실제 수행된 시점의 xid - vacuum_freeze_min_age 값을 가짐)

